# Test z tutoriala
https://medium.com/@alyafey22/sentiment-classification-from-keras-to-the-browser-7eda0d87cdc6

In [67]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from scipy.spatial.distance import cdist

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import save_model
import csv 
import pandas as pd 
from IPython.display import display, HTML
import h5py
from random import shuffle

import re

In [98]:
def load_dataset(file = './dane_treningowe.csv'):
    with open(file, 'r', encoding='utf8') as f:
        labels = []
        text = []

        lines = f.readlines()
    shuffle(lines)
    for line in lines:
        data = line.split(',')
        if len(data) == 2:
            labels.append(data[1])
            text.append(data[0])
    return text,labels

In [110]:
x_train_text, y_train = load_dataset()
data_text = x_train_text
idx = 5
print(x_train_text[idx],'\n',y_train[idx])

@briannaxoxx you are soo weird!  
 1



In [87]:
def tokenize(thresh = 5):
    count  = dict()
    idx = 1
    word_index = dict()
    for txt in data_text:
        words = process(txt)
        for word in words:
            if word in count.keys():
                count[word] += 1
            else:
                count[word]  = 1
    most_counts = [word for word in count.keys() if count[word]>=thresh]
    for word in most_counts:
        word_index[word] = idx
        idx+=1
    return word_index

In [88]:
def process(txt):
    out = re.sub(r'[^a-zA-Z0-9\s]', '', txt)
    out = out.split()
    out = [word.lower() for word in out]
    return out

In [89]:
def getMax(data):
    max_tokens = 0 
    for txt in data:
        if max_tokens < len(txt.split()):
            max_tokens = len(txt.split())
    return max_tokens

In [73]:
def create_sequences(data):
    tokens = []
    for txt in data:
        words = process(txt)
        seq = [0] * max_tokens
        i = 0 
        for word in words:
            start = max_tokens-len(words)
            if word.lower() in word_index.keys():
                seq[i+start] = word_index[word]
            i+=1
        tokens.append(seq)        
    return np.array(tokens)

In [90]:
num_wordsnum_wor  = None
word_index = tokenize()
num_words = len(word_index)+1
print('length of the dictionary ',len(word_index))
max_tokens = getMax(x_train_text)

length of the dictionary  9125


In [91]:
x_train_tokens = create_sequences(x_train_text)

In [92]:
model = Sequential()
embedding_size = 8
model.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_tokens,
                    name='layer_embedding'))

model.add(GRU(units=16, name = "gru_1",return_sequences=True))
model.add(GRU(units=8, name = "gru_2" ,return_sequences=True))
model.add(GRU(units=4, name= "gru_3"))
model.add(Dense(1, activation='sigmoid',name="dense_1"))
optimizer = Adam(lr=1e-4)
model.compile(loss='binary_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

In [93]:
y_train_int = [int(i) for i in y_train]

In [94]:
model.fit(x_train_tokens, y_train_int, validation_split=0.05, epochs=5, batch_size=32)

Train on 63946 samples, validate on 3366 samples
Epoch 1/20
63946/63946 [==============================] - 172s 3ms/step - loss: 0.6800 - acc: 0.5710 - val_loss: 0.6700 - val_acc: 0.5870
Epoch 2/20
63946/63946 [==============================] - 169s 3ms/step - loss: 0.6493 - acc: 0.6214 - val_loss: 0.6319 - val_acc: 0.6453
Epoch 3/20
63946/63946 [==============================] - 172s 3ms/step - loss: 0.6107 - acc: 0.6652 - val_loss: 0.6023 - val_acc: 0.6702
Epoch 4/20
63946/63946 [==============================] - 173s 3ms/step - loss: 0.5775 - acc: 0.6982 - val_loss: 0.5789 - val_acc: 0.6916
Epoch 5/20
39264/63946 [=================>............] - ETA: 1:06 - loss: 0.5536 - acc: 0.7184

KeyboardInterrupt: 

In [97]:
txt = ["fuck Donald Trump", "Harry Potter", "hell", "cat"]
print(create_sequences(txt)[0])
pred = model.predict(create_sequences(txt))
print('\n prediction for \n',pred[:,0])

[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0 1311
    0    0]

 prediction for 
 [0.58993304 0.6694414  0.66111654 0.72955984]
